# Probabilistic State Estimation

> This tutorial is implementation for [MIT state estimation class](https://ocw.mit.edu/courses/6-01sc-introduction-to-electrical-engineering-and-computer-science-i-spring-2011/pages/unit-4-probability-and-planning/state-estimation/) &
[this](https://automaticaddison.com/how-to-derive-the-state-space-model-for-a-mobile-robot/)

In [ ]:
#| default_exp probabilistic_state_estimation

In [ ]:
#| hide
import random

In [ ]:
#| export
class DDist:
    def __init__(self, dictionary:dict) -> None:
        self.d = dictionary
    def prob(self, elt # an element of the domain of this distribution
             ) -> float:
        if elt in self.d:
            return self.d[elt]
        else:
            return 0
    def support(self):
        return [k for k in self.d.keys() if self.prob(k) > 0]
    @property
    def dist(self):
        return self.d
    def draw(self):
        r = random.random()
        sum = 0.0
        for val in self.support():
            sum += self.prob(val)
            if r < sum:
                return val

## Conditional Probabilities

In [ ]:
def TgivenD(D):
    if D == 'disease':
        return DDist({'positive' : 0.99, 'negative' : 0.01})
    elif D == 'nodisease':
        return DDist({'positive' : 0.001, 'negative' : 0.999})
    else:
        raise Exception('invalid value for D')

In [ ]:
TgivenD('disease').prob('negative')

0.01

## Joint Probability Distribution

> Excercise to estimate the joint distribution probability given $P(A)$ and $P(B|A)$.

In [ ]:
#| hide
# P(B | A)
def PBgA(a:DDist) -> DDist:
    if a == 'a1':
        return DDist({'b1' : 0.7, 'b2' : 0.3})
    else:
        return DDist({'b1' : 0.2, 'b2' : 0.8})
# P(A)
PA = DDist({'a1' : 0.9, 'a2' : 0.1})
print('PA.prob({}): {}'.format('a1', PA.prob('a1')))

# P(B | A = a1)
print('PBgA({}).prob({}): {}'.format('a1', 'b1', PBgA('a1').prob('b1')))

PA.prob(a1): 0.9
PBgA(a1).prob(b1): 0.7


In [ ]:
#| export
# Calculate the join distribution probabilities.
class JDist(DDist):
    def __init__(self, PA:DDist # P(A)
                 ,PBgA:callable # condtional probability P(B|A)
                 ) -> DDist:
        PAB = dict()
        for a in PA.dist.keys():
            PBgA_a = PBgA(a)
            for b in PBgA_a.dist.keys():
                PAB[(a,b)] = PA.prob(a) * PBgA_a.prob(b)
        self.d = PAB
    def marginalizeOut(self, 
                       idx:int # which random variable ..?
                       ) -> DDist:
        # A = a
        symbolKeys = self.d.keys()
        for key in symbolKeys:
            s = key[idx]
            
        return
    

In [ ]:
JDist(PA, PBgA).dist

{('a1', 'b1'): 0.63,
 ('a1', 'b2'): 0.27,
 ('a2', 'b1'): 0.020000000000000004,
 ('a2', 'b2'): 0.08000000000000002}

## Marginalization
> Marginalize out A or B

# State Space Model

$$ X_{t} = \begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} = \begin{bmatrix} x_{t-1} + v_{t-1} cos(\gamma_{t-1}) * dt \\ y_{t-1} + v_{t-1} sin(\gamma_{t-1}) * dt \\ \gamma_{t-1} + \omega_{t-1} * dt \end{bmatrix} = \begin{bmatrix} f_{1} \\ f_{2} \\ f_{3} \end{bmatrix} $$

where

$$
X_{t-1} = \begin{bmatrix} x_{t-1} \\ y_{t-1} \\ \gamma_{t-1} \end{bmatrix} 
$$

$$
\cos(\gamma_{t-1}) = \frac{v_{x_{t-1}}}{v_{t-1}}
$$

$$
\sin(\gamma_{t-1}) = \frac{v_{y_{t-1}}}{v_{t-1}}
$$

$$
\begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} = A_{t-1} \begin{bmatrix} x_{t-1} \\ y_{t-1} \\ \gamma_{t-1} \end{bmatrix} + B_{t-1} \begin{bmatrix} v_{t-1} \\ \omega_{t-1} \end{bmatrix}
$$

***How to Calculate the A Matrix***

Our system is expressed as a nonlinear system now because the state is a function of cosines and sines (which are nonlinear trigonometric operations).
to “linearize” the nonlinear equations. To do this, we need to calculate the Jacobian, which is nothing more than a fancy name for **matrix of partial derivatives.**


$$ 
A_{t-1} = \begin{bmatrix}
\frac{\partial f_{1}}{\partial x_{t-1}} & \frac{\partial f_{1}}{\partial y_{t-1}} & \frac{\partial f_{1}}{\partial \gamma_{t-1}} \\
\frac{\partial f_{2}}{\partial x_{t-1}} & \frac{\partial f_{2}}{\partial y_{t-1}} & \frac{\partial f_{2}}{\partial \gamma_{t-1}} \\
\frac{\partial f_{3}}{\partial x_{t-1}} & \frac{\partial f_{3}}{\partial y_{t-1}} & \frac{\partial f_{3}}{\partial \gamma_{t-1}} \\
\end{bmatrix}
$$

where

$$
\frac{\partial f_{1}}{\partial x_{t-1}} = \frac{\partial(x_{t-1} + v_{t-1} cos(\gamma_{t-1}) * dt)}{\partial x_{t-1}} = \frac{\partial (x_{t-1})}{\partial x_{t-1}} +  \frac{\partial (v_{t-1} cos(\gamma_{t-1}) * dt)}{\partial x_{t-1}} = 1 + 0 = 1
$$

Hence for the rest of the 9 partial derivatives, you will get $A_{t-1} = I_{3}$, ***This is due to the fact that robot on wheels only drives when the wheels are commanded to turn.***

***How to Calculate the B Matrix***

The B matrix has **the same number of rows as the number of states and has the same number of columns as the number of control inputs.**

The control inputs in this example are the linear velocity (v) and the angular velocity around the z axis, ω (also known as the “yaw rate”).

The B matrix expresses how the state of the system (i.e. [x,y,γ]) changes from t-1 to t due to the control commands (i.e. control inputs v and ω)

$$
B_{t-1} = \begin{bmatrix}
\frac{\partial f_{1}}{\partial v_{t-1}} & \frac{\partial f_{1}}{\partial \omega_{t-1}} \\
\frac{\partial f_{2}}{\partial v_{t-1}} & \frac{\partial f_{2}}{\partial \omega_{t-1}} \\
\frac{\partial f_{3}}{\partial v_{t-1}} & \frac{\partial f_{3}}{\partial \omega_{t-1}} \\
\end{bmatrix} = \begin{bmatrix}
cos(\gamma_{t-1}) * dt & 0 \\
sin(\gamma_{t-1}) * dt & 0 \\
0 & dt\\
\end{bmatrix}
$$

***Putting It All Together***

$$
\begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} = A_{t-1} \begin{bmatrix} x_{t-1} \\ y_{t-1} \\ \gamma_{t-1} \end{bmatrix} + B_{t-1} \begin{bmatrix} v_{t-1} \\ \omega_{t-1} \end{bmatrix}
$$

$$
\begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} = I_{3} \begin{bmatrix} x_{t-1} \\ y_{t-1} \\ \gamma_{t-1} \end{bmatrix} + \begin{bmatrix}
cos(\gamma_{t-1}) * dt & 0 \\
sin(\gamma_{t-1}) * dt & 0 \\
0 & dt\\
\end{bmatrix} \begin{bmatrix} v_{t-1} \\ \omega_{t-1} \end{bmatrix}
$$

> Additive noise can be added to each state as $noise_{t-1}$

In [ ]:
#| export
import numpy as np

def calculate_B_matrix(yaw:float, dt:float=0.1) -> np.array:
     """
     Calculates and returns the `B` matrix.

     A 3x2 matrix (number of states x number of control inputs).
     The control inputs are the forward speed and the
     rotation rate around the z axis from the x-axis in the 
     counterclockwise direction.

     [v, yaw_rate]

     This matrix expresses how the state of the system [x,y,yaw] changes
     from t-1 to t due to the control commands (i.e. control input).

     Parameters
     ----------
     yaw
          The yaw (rotation angle around the z axis) in radians.
     dt
          The change in time from time step t-1 to t in seconds.

     Returns
     -------
     np.ndarray: The B matrix as a NumPy array.
     """
     B = np.array([
          [np.cos(yaw) * dt, 0],
          [np.sin(yaw) * dt, 0],
          [0, dt]
     ])
     return B

In [ ]:
# Author: Addison Sears-Collins
# https://automaticaddison.com
# Description: A state space model for a differential drive mobile robot

# A matrix
# 3x3 matrix -> number of states x number of states matrix
# Expresses how the state of the system [x,y,yaw] changes 
# from t-1 to t when no control command is executed.
# Typically a robot on wheels only drives when the wheels are commanded
# to turn.
# For this case, A is the identity matrix.
# A is sometimes F in the literature.
A_t_minus_1 = np.eye(3)

# The estimated state vector at time t-1 in the global 
# reference frame
# [x_t_minus_1, y_t_minus_1, yaw_t_minus_1]
# [meters, meters, radians]
state_estimate_t_minus_1 = np.array([0.0, 0.0, 0.0])

# The control input vector at time t-1 in the global 
# reference frame
# [v, yaw_rate]
# [meters/second, radians/second]
# In the literature, this is commonly u.
control_vector_t_minus_1 = np.array([4.5, 0.05])

process_noise_v_t_minus_1 = np.array([0.01, 0.01, 0.003])
process_noise_v_t_minus_1 = np.array([0.01, 0.01, 0.003])

yaw_angle = 0.0  # radians
dt = 1.0  # seconds

def main():
    """
    Main function to calculate and print the state estimate at time t.
    """
    state_estimate_t = (
        A_t_minus_1 @ state_estimate_t_minus_1 +
        calculate_B_matrix(yaw_angle, dt) @ control_vector_t_minus_1 +
        process_noise_v_t_minus_1
    )

    print(f'State at time t-1: {state_estimate_t_minus_1}')
    print(f'Control input at time t-1: {control_vector_t_minus_1}')
    print(f'State at time t: {state_estimate_t}')  # State after dt seconds

if __name__ == "__main__":
    main()


State at time t-1: [0. 0. 0.]
Control input at time t-1: [4.5  0.05]
State at time t: [4.51  0.01  0.053]


# Observation Model

An observation model (also known as measurement model or sensor model) is a mathematical equation that represents a vector of predicted sensor measurements y at time t as a function of the state of a robotic system x at time t, plus some sensor noise (because sensors aren’t 100% perfect) at time t, denoted by vector $w_{t}$.

$$
    y_{t} = Hx_{t} + w_{t}
$$

where

$$
\begin{bmatrix} y_{1}^{t} \\ y_{2}^{t} \\ \vdots \\ y_{n}^{t} \end{bmatrix} = H^{t}_{nx^{3}} \begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} + \begin{bmatrix} w_{1} \\ w_{2} \\ \vdots \\ w_{n} \end{bmatrix}
$$

***How to Calculate the H Matrix***

In this case, H will be the identity matrix since the estimated state maps directly to sensor measurements [x, y, yaw]. 

H has the same number of rows as **sensor measurements** and the same number of columns as **states**.  $H_{t} = I_{3}$

What if, for example, you had a landmark in your environment. How does the current state of the robot enable you to calculate the distance (i.e. range) to the landmark r and the bearing angle b to the landmark?

<p align="center">
  <img src="https://automaticaddison.com/wp-content/uploads/2020/12/observation-model-7JPG.jpg" alt="landmark">
</p>

Using the Pythagorean Theorem and trigonometry, we get the following equations for the range r and bearing b:

$$
 r = \sqrt{(x_{t} - x_{landmark})^2 + (y_{t} - y_{landmark})^2} \\
 b = \mathrm{atan2}(y_{t} - y_{landmark}, x_{t} - x_{landmark}) - \gamma_{t}
$$

Let’s put this in matrix form.

$$
\begin{bmatrix} r \\ b \end{bmatrix} = \begin{bmatrix} h_{1} \\ h_{2} \end{bmatrix} = \begin{bmatrix} \sqrt{(x_{t} - x_{landmark})^2 + (y_{t} - y_{landmark})^2} \\ \mathrm{atan2}(y_{t} - y_{landmark}, x_{t} - x_{landmark}) - \gamma_{t} \end{bmatrix}
$$

Let’s linearize the model to create a linear observation model of the following form:

$$
\begin{bmatrix} y_{1}^{t} \\ y_{2}^{t} \\ \vdots \\ y_{n}^{t} \end{bmatrix} = H^{t}_{nx^{3}} \begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} + \begin{bmatrix} w_{1} \\ w_{2} \\ \vdots \\ w_{n} \end{bmatrix}
$$

$$
H_{t} = \begin{bmatrix}
\frac{\partial r}{\partial x_{t}} & \frac{\partial r}{\partial y_{t}} & \frac{\partial r}{\partial \gamma_{t}} \\
\frac{\partial b}{\partial x_{t}} & \frac{\partial b}{\partial y_{t}} & \frac{\partial b}{\partial \gamma_{t}} \\
\end{bmatrix} = \begin{bmatrix}
\frac{x_{t} - x_{L}}{ \sqrt{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2}} & \frac{y_{t} - y_{L}}{ \sqrt{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2}} & 0 \\
\frac{-(y_{t} - y_{L})}{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2} & \frac{(x_{t} - x_{L})}{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2} & -1 \\
\end{bmatrix} 
$$

$$
\begin{bmatrix} y_{1}^{t}\end{bmatrix} = \begin{bmatrix}
\frac{x_{t} - x_{L}}{ \sqrt{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2}} & \frac{y_{t} - y_{L}}{ \sqrt{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2}} & 0 \\
\frac{-(y_{t} - y_{L})}{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2} & \frac{(x_{t} - x_{L})}{(x_{t} - x_{L})^2 + (y_{t} - y_{L})^2} & -1 \\
\end{bmatrix} \begin{bmatrix} x_{t} \\ y_{t} \\ \gamma_{t} \end{bmatrix} + \begin{bmatrix} w_{r} \\ w_{b} \end{bmatrix}
$$

Equivalently, in some literature, you might see that all the stuff above is equal to:

$$
h(\hat{x}_{t|t-1})
$$

In [ ]:
# Author: Addison Sears-Collins
# https://automaticaddison.com
# Description: An observation model for a differential drive mobile robot
 
# Measurement matrix H_t
# Used to convert the predicted state estimate at time t
# into predicted sensor measurements at time t.
# In this case, H will be the identity matrix since the 
# estimated state maps directly to state measurements from the 
# odometry data [x, y, yaw]
# H has the same number of rows as sensor measurements
# and same number of columns as states.
H_t = np.array([    [1.0,  0,   0],
                    [  0,1.0,   0],
                    [  0,  0, 1.0]])
                 
# Sensor noise. This is a vector with the
# number of elements equal to the number of sensor measurements.
sensor_noise_w_t = np.array([0.07,0.07,0.04])
                         
# The estimated state vector at time t in the global 
# reference frame
# [x_t, y_t, yaw_t]
# [meters, meters, radians]
state_estimate_t = np.array([5.2,2.8,1.5708])
 
def main():
 
    estimated_sensor_observation_y_t = H_t @ (
            state_estimate_t) + (
            sensor_noise_w_t)
 
    print(f'State at time t: {state_estimate_t}')
    print(f'Estimated sensor observations at time t: {estimated_sensor_observation_y_t}')
 
main()

State at time t: [5.2    2.8    1.5708]
Estimated sensor observations at time t: [5.27   2.87   1.6108]


# The Jacobian

Consider the following non-linear function:

$$ 
f(x,y) = \begin{bmatrix} sin(x) + y^2 \\ cos(y) + x^2 \end{bmatrix} = \begin{bmatrix} f_{1} \\ f_{2} \end{bmatrix}
$$

The Jacobian matrix of this function is a 2x2 matrix containing the partial derivatives of each output with respect to each input.

$$ 
J = \begin{bmatrix}  \frac{\partial  f_{1}}{\partial x} && \frac{\partial f_{1}}{\partial y} \\ \frac{\partial f_{2}}{\partial y}  && \frac{\partial f_{2}}{\partial y} \end{bmatrix} \\[10pts]

\frac{\partial  f_{1}}{\partial x} =  \frac{\partial  (sin(x) + y^2)}{\partial x} = \frac{\partial (sin(x))}{{\partial x}} + \frac{\partial (y^2)}{\partial x} = cos(x) + 0 = cos(x) \\[10pts]

\frac{\partial  f_{1}}{\partial y} =  \frac{\partial  (sin(x) + y^2)}{\partial y} = \frac{\partial (sin(x))}{{\partial y}} + \frac{\partial (y^2)}{\partial y} = 0 + 2y = 2y \\[10pts]

\frac{\partial  f_{2}}{\partial x} =  \frac{\partial  (cos(y) + x^2)}{\partial x} = \frac{\partial (cos(y))}{{\partial x}} + \frac{\partial (x^2)}{\partial x} = 0 + 2x = 2x \\[10pts]

\frac{\partial  f_{2}}{\partial y} =  \frac{\partial  (cos(y) + x^2)}{\partial y} = \frac{\partial (cos(y))}{{\partial y}} + \frac{\partial (x^2)}{\partial y} = -sin(y) + 0 = -sin(y) \\[10pts]

J = \begin{bmatrix}  cos(x) && 2y \\ 2x && -sin(y) \end{bmatrix} \\[10pts]
$$

### Symbolic Computation of the Jacobian

In [3]:
#| export
import sympy as sp

# Define the symbols
x, y = sp.symbols('x y')

# Define the function
f1 = sp.sin(x) + y**2
f2 = sp.cos(y) + x**2
f = sp.Matrix([f1, f2])

# Compute the Jacobian matrix
J = f.jacobian([x, y])

# Print the Jacobian matrix
print("Symbolic Jacobian matrix:")
sp.pprint(J)
sp.pprint(f)


Symbolic Jacobian matrix:
⎡cos(x)    2⋅y  ⎤
⎢               ⎥
⎣ 2⋅x    -sin(y)⎦
⎡ 2         ⎤
⎢y  + sin(x)⎥
⎢           ⎥
⎢ 2         ⎥
⎣x  + cos(y)⎦


In [ ]:
x, y, xm, ym, xl, yl = sp.symbols('x y x_m y_m x_l y_l')

xdiff = sp.sq 

In [42]:
import sympy as sp

# Define the symbols
x, y, x_m, y_m, x_l, y_l, theta = sp.symbols('x y x_m y_m x_l y_l theta')

# Define the function for the difference in x
r_m = sp.sqrt((x - x_m)**2 + (y - y_m)**2)
phi_m = sp.atan2(y - y_m, x - x_m) - theta
xdiff_m = x + r_m * sp.cos(theta + phi_m)

r_l = sp.sqrt((x - x_l)**2 + (y - y_l)**2)
phi_l = sp.atan2(y - y_l, x - x_l) - theta
xdiff_l = x + r_l * sp.cos(theta + phi_l)

# Define the function for the difference in y
ydiff_m = y + r_m * sp.sin(theta + phi_m)
ydiff_l = y + r_l * sp.sin(theta + phi_l)


# Combine the terms
xdiff = (xdiff_m - xdiff_l)**2

ydiff = (ydiff_m - ydiff_l)**2

f = sp.Matrix([xdiff, ydiff])

# Compute the Jacobian matrix
J = f.jacobian([x, y, theta])

# Print the Jacobian matrix
print("Symbolic Jacobian matrix:")
sp.pprint(J)
sp.pprint(f)

Symbolic Jacobian matrix:
⎡0  0  0⎤
⎢       ⎥
⎣0  0  0⎦
⎡         2⎤
⎢(xₗ - xₘ) ⎥
⎢          ⎥
⎢         2⎥
⎣(yₗ - yₘ) ⎦


In [ ]:
import numpy as np

def f(vec):
    x, y = vec
    return np.array([np.sin(x) + y**2, np.cos(y) + x**2])

def jacobian(f, vec, eps=1e-5):
    n = len(vec)
    jac = np.zeros((n, n))
    f0 = f(vec)
    for i in range(n):
        vec_eps = np.copy(vec)
        vec_eps[i] += eps
        f1 = f(vec_eps)
        jac[:, i] = (f1 - f0) / eps
    return jac

# Example input
vec = np.array([1.0, 2.0])

# Compute the Jacobian matrix numerically
J_num = jacobian(f, vec)

print("Numerical Jacobian matrix:")
print(J_num)

Numerical Jacobian matrix:
[[ 0.5402981   4.00001   ]
 [ 2.00001    -0.90929535]]


In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()